In [51]:
# IMPORTS

# Base
import json
import pandas as pd

# Inner
from scripts.file_manager import YAML_read, JSON_load, JSON_open
from scripts.requests import Request_post


In [52]:
# Get settings
file = YAML_read('config')
file.get_config()
token = file.config['service_token'] # Exclude API token
link = file.config['service_link'] # Exclude API url
json_o = JSON_open('data')
json_o.open()
start_data = json_o.data
actual_data_id = start_data[0]['id'] # This is maximum data id in offline dataset

In [53]:
# Get data on cycle request 

result = Request_post(link, token) # First request
data = []
snap = result.result
num = 1
work = True
while work:
    if snap['next']:
        for result in snap['results']:
            if result['id'] > actual_data_id:
                data += [result]
                link = snap['next']
                result = Request_post(link, token)
                snap = result.result
                num += 1
            else:
                print('Actual data not found')
                work = False
                break
        
    else:
        work = False


Actual data not found


In [54]:
# Save data
data = data+start_data
json_l = JSON_load('data')
json_l = json_l.load(data)

In [72]:
# Pandas
df=pd.DataFrame(data)
ids = list(df['identity'].unique())
users_count = len(df['identity'].unique())
print(f'Количество пользователей - {users_count}')
df['date'] = df.apply(lambda x: x['time_create'].split('T')[0],axis=1)
df.head()

Количество пользователей - 210


,id,identity,command,time_create,date
0,2143,315854463,progress_words,2022-12-21T18:32:39.305321+03:00,2022-12-21
1,2142,409308393,open_settings,2022-12-21T15:13:22.662603+03:00,2022-12-21
2,2141,315854463,exit,2022-12-21T15:04:03.171655+03:00,2022-12-21
3,2140,315854463,new_words,2022-12-21T15:03:55.157946+03:00,2022-12-21
4,2139,315854463,exit,2022-12-21T15:03:53.964027+03:00,2022-12-21


In [57]:
def get_days(row):
    try:
        return len(list(df[df['identity'] == row['identity']]['date'].unique()))
    except:
        0
        
def get_counts(row):
    return df['identity'].value_counts ()[row['identity']]

# Data days, counts
days_stat = pd.DataFrame(ids, columns=['identity'])
days_stat['days'] = days_stat.apply(get_days, axis=1)
days_stat['counts'] = days_stat.apply(get_counts, axis=1)
days_stat.sort_values(by='days', ascending=False).reset_index()[['identity', 'days', 'counts']].head(10)

,identity,days,counts
0,315854463,20,710
1,409308393,13,70
2,5876254329,9,88
3,5588561064,4,25
4,5141647366,4,16
5,336839126,3,12
6,164682186,3,23
7,1518539728,3,31
8,941727774,3,12
9,348958472,2,9


In [73]:
# Data commands
commands = df['command'].value_counts().reset_index()
commands.columns = ['command', 'counts']
commands.head(10)

,command,counts
0,exit,357
1,new_words,330
2,/start,256
3,/menu,145
4,open_settings,111
5,repeat_words,110
6,grammar_book,103
7,proceed_start,89
8,main_menu,73
9,progress_words,69
